In [2]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [4]:
includet("utils.jl")
includet("turing_nw_bvar.jl")

In [5]:
df = DataFrame(XLSX.readtable("../../data/usa_world_data.xlsx", "Arkusz1", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

146×15 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-03-01 to 2024-04-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1988-03-01 │   0.0857 │   677.401 │       262.6 │  9.04333 │  18.59 │ 14.433 ⋯
│ 1988-06-01 │   0.0882 │   681.206 │       269.1 │  9.30667 │  21.67 │ 14.608 ⋯
│ 1988-09-01 │   0.0887 │   694.387 │       277.4 │     9.55 │  22.73 │ 14.811 ⋯
│ 1988-12-01 │   0.0914 │   697.568 │       285.4 │  9.81333 │  23.75 │ 15.088 ⋯
│ 1989-03-01 │    0.093 │   706.377 │       302.3 │  10.0867 │  24.96 │ 15.686 ⋯
│ 1989-06-01 │    0.081 │    780.73 │       331.9 │  10.4233 │  25.22 │ 17.013 ⋯
│ 1989-09-01 │   0.0831 │   785.343 │       347.4 │  10.7967 │  23.69 │ 17.640 ⋯
│ 1989-12-01 │   0.0793 │   821.683 │      339.97 │    11.14 │  22.87 │ 17.048 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [65]:
freq = 4
#:Price_10Y
returns = percentchange(data_source[:USA_Net, :Price_10Y, :World_ex_USA, :EM_Net], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
#excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
#data = merge(real_returns, real_t_bill, term, cpi, dy)
data = real_returns

start = Date(1989, 01,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d))))

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1466,0.9025,0.096,0.283,-1.4498,-1.2239,-0.0302,0.131,5.1481,0.6551,0.1176
Price_10Y,0.0836,0.5691,0.0533,0.1694,-0.2899,0.3136,-0.0702,0.0162,0.0266,0.6377,0.0313
World_ex_USA,0.1676,1.2168,0.0465,0.2624,-1.7796,-0.9618,-0.1504,0.124,5.7703,0.2772,0.0755
EM_Net,0.253,1.7838,0.0802,0.3814,-2.3616,-0.8653,-0.1935,0.0758,4.3012,0.3168,0.1287


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net_CPI,0.1449,0.8631,0.0693,0.2568,-1.3483,-1.0427,-0.0548,0.1149,3.9088,0.4783,0.0943
Price_10Y_CPI,0.0893,0.6707,0.0266,0.1468,-0.3841,0.3686,-0.1026,0.0313,0.5634,0.2975,0.0121
World_ex_USA_CPI,0.1641,1.1774,0.0197,0.226,-1.678,-0.7637,-0.1656,0.1193,4.7076,0.1204,0.0465
EM_Net_CPI,0.2492,1.7444,0.0534,0.3523,-2.26,-0.765,-0.2243,0.0713,3.8365,0.2144,0.0663


nothing

"Correlations"

4×4 Matrix{Float64}:
  1.0       -0.316058   0.782936   0.627688
 -0.316058   1.0       -0.357511  -0.405884
  0.782936  -0.357511   1.0        0.716905
  0.627688  -0.405884   0.716905   1.0

In [66]:
s_data

140×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1989-03-01 to 2023-12-01
┌────────────┬─────────────┬───────────────┬──────────────────┬─────────────┐
│            │ USA_Net_CPI │ Price_10Y_CPI │ World_ex_USA_CPI │ EM_Net_CPI  │
├────────────┼─────────────┼───────────────┼──────────────────┼─────────────┤
│ 1989-03-01 │   0.0400226 │   -0.00383119 │        0.0132582 │    0.070071 │
│ 1989-06-01 │   0.0998849 │     0.0895742 │       -0.0730667 │    0.270879 │
│ 1989-09-01 │   0.0869226 │   -0.00370785 │        0.0482182 │  -0.0531522 │
│ 1989-12-01 │  -0.0212672 │     0.0310039 │        0.0366651 │   0.0698423 │
│ 1990-03-01 │  -0.0442563 │    -0.0389844 │       -0.0854084 │    0.173533 │
│ 1990-06-01 │   0.0771754 │     0.0244056 │       -0.0217388 │ -0.00917896 │
│ 1990-09-01 │    -0.12709 │    -0.0278514 │        -0.117558 │  -0.0423492 │
│ 1990-12-01 │  0.00214804 │     0.0616911 │       -0.0739193 │   -0.218053 │
│     ⋮      │      ⋮      │       ⋮       │        ⋮         │   

In [104]:
T, K = size(real_returns)

model = mvnormal_lkjcholesky_model(values(real_returns), K, T)

# Sample from the posterior
chain = sample(model, NUTS(), MCMCThreads(), 1000, 4; discard_initial=1000)

# Summarize results
display(summarystats(chain))
display(quantile(chain))


Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.003125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.00625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
Sampling (4 threads)  25%|████████                      |  ETA: 0:00:30
Sampling (4 threads)  50%|████████████████              |  ETA: 0:00:19
Sampling (4 threads)  75%|███████████████████████       |  ETA: 0:00:07
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:00:23
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:00:23


Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        μ[1]    0.0176    0.0061    0.0001   3225.6424   2946.6567    0.9999   ⋯
        μ[2]    0.0059    0.0037    0.0001   4900.0320   3480.0746    1.0005   ⋯
        μ[3]    0.0062    0.0071    0.0001   2837.0790   3165.7380    1.0005   ⋯
        μ[4]    0.0142    0.0105    0.0002   3226.5457   3238.7891    1.0009   ⋯
        σ[1]    0.0721    0.0042    0.0001   2377.1601   2475.0064    1.0021   ⋯
        σ[2]    0.0447    0.0026    0.0000   4059.7745   3311.5389    1.0006   ⋯
        σ[3]    0.0824    0.0049    0.0001   2177.2875   2620.3075    1.0018   ⋯
        σ[4]    0.1240    0.0075    0.0001   2649.5771   2730.4195    1.0009   ⋯
   L.L[1, 1]    1.0000    0.0000       NaN         NaN         NaN       NaN   ⋯
   L.L[2, 1]   -0.3008    0.0755    0.0012   3736.6800   3000.4136    1.0008   ⋯
   L.L[3

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

        μ[1]    0.0056    0.0135    0.0177    0.0216    0.0294
        μ[2]   -0.0016    0.0034    0.0059    0.0085    0.0129
        μ[3]   -0.0077    0.0013    0.0062    0.0111    0.0200
        μ[4]   -0.0062    0.0067    0.0144    0.0215    0.0345
        σ[1]    0.0645    0.0692    0.0718    0.0748    0.0810
        σ[2]    0.0400    0.0429    0.0446    0.0464    0.0501
        σ[3]    0.0732    0.0790    0.0823    0.0857    0.0926
        σ[4]    0.1109    0.1185    0.1235    0.1290    0.1396
   L.L[1, 1]    1.0000    1.0000    1.0000    1.0000    1.0000
   L.L[2, 1]   -0.4412   -0.3529   -0.3055   -0.2498   -0.1435
   L.L[3, 1]    0.6875    0.7391    0.7638    0.7862    0.8246
   L.L[4, 1]    0.4778    0.5604    0.5988    0.6339    0.6942
   L.L[2, 2]    0.8974    0.9357    0.9522    0.9683    0.9897
   L.L[3, 2]   -0.2222   -0.1519   -0.1159

In [122]:
μ = mean(group(chain, :μ)).nt.mean
σ = mean(group(chain, :σ)).nt.mean
L = mean(group(chain, "L.L")).nt.mean

Σ = est_covariance(σ, L)

n_samples = 10000
n_steps = 25 * freq
scenarios = zeros(K, n_steps, n_samples)

for t in 1:n_steps
    scenarios[:,t,:]= rand(MvNormal(μ, Σ), 10000)
end

#= for k in 1:K
    display(round.(quantile(scenarios[k,1,:], [.025, .25, .5, .75, .975]), digits=2))
end =#

ret_in_years = cum_returns_in_periods(scenarios, [1,5, 15, 25], freq, true)
print_scenarios_summary(ret_in_years)
assets = ["USA", "Bonds", "World", "EM"]

for a in 1:4
    print_scenarios_percentiles(ret_in_years[a,:,:], [.03, .25, .5, .75, .97], assets[a])
end 

,USA,Bonds,World,EM
1,0.0736,0.0232,0.0286,0.0598
5,0.0711,0.0236,0.0259,0.0576
10,0.0705,0.0235,0.0248,0.0569
25,0.0707,0.0235,0.025,0.0567


,USA,Bonds,World,EM
1,0.1451,0.0903,0.1659,0.2471
5,0.0652,0.0401,0.0739,0.111
10,0.0377,0.023,0.0429,0.0636
25,0.0292,0.0179,0.0332,0.0501


,USA,Bonds,World,EM
1,-0.0029,0.0089,-0.0286,-0.0311
5,-0.064,-0.003,-0.018,-0.0375
10,-0.0162,-0.0088,0.0279,0.0054
25,-0.0035,-0.0236,0.037,0.0167


,USA,Bonds,World,EM
1,0.0726,-0.0079,0.0556,-0.042
5,0.1576,-0.0654,0.0868,0.0796
10,-0.0893,0.0235,-0.1046,-0.0328
25,0.0322,0.003,0.0127,0.0578


,0.03,0.25,0.5,0.75,0.97
1,-0.1988,-0.0227,0.0739,0.1704,0.3459
2,-0.0522,0.0286,0.0715,0.1145,0.1931
3,-0.0003,0.0447,0.0708,0.0964,0.1413
4,0.0152,0.051,0.0705,0.0904,0.1246


,0.03,0.25,0.5,0.75,0.97
1,-0.1488,-0.0365,0.0225,0.0841,0.1931
2,-0.0517,-0.0034,0.0238,0.0505,0.0986
3,-0.0197,0.008,0.0234,0.039,0.0664
4,-0.0101,0.0116,0.0236,0.0355,0.0572


,0.03,0.25,0.5,0.75,0.97
1,-0.2843,-0.0824,0.0308,0.1401,0.3391
2,-0.1117,-0.0241,0.0257,0.0761,0.1637
3,-0.055,-0.0046,0.0244,0.0537,0.1061
4,-0.0365,0.0026,0.0247,0.0471,0.0878


,0.03,0.25,0.5,0.75,0.97
1,-0.4038,-0.1072,0.0616,0.229,0.5166
2,-0.1505,-0.0154,0.0578,0.1324,0.2647
3,-0.0624,0.0139,0.0568,0.1001,0.176
4,-0.0383,0.023,0.057,0.0905,0.151


In [119]:
#= for a in 1:4
    print_scenarios_percentiles(ret_in_years[a,:,:], [.03, .25, .5, .75, .97])
end =#

print_scenarios_percentiles(ret_in_years[1,:,:], [.03, .25, .5, .75, .97])


,1,2,3,4
0.03,-0.187,-0.0415,0.0096,0.025
0.25,-0.017,0.0367,0.056,0.0612
0.5,0.0806,0.0809,0.0815,0.0812
0.75,0.1821,0.1264,0.1068,0.1014
0.97,0.3712,0.2059,0.153,0.1374


In [118]:
ret_in_years = cum_returns_in_periods(expm1.(scenarios), [1,5, 15, 25], freq, true)
print_scenarios_summary(ret_in_years)

,USA,Bonds,World,EM
1,0.0847,0.0267,0.0408,0.0916
5,0.0814,0.0276,0.0388,0.0886
10,0.0814,0.0276,0.0384,0.0877
25,0.0813,0.0277,0.0381,0.0875


,USA,Bonds,World,EM
1,0.149,0.0896,0.1662,0.25
5,0.0663,0.0404,0.0743,0.1112
10,0.0382,0.023,0.0428,0.0645
25,0.03,0.0178,0.0335,0.0501


,USA,Bonds,World,EM
1,0.1213,0.0825,0.138,0.1776
5,0.0504,0.0212,0.0479,0.0593
10,0.014,0.0256,0.0406,0.0357
25,0.0119,0.0101,0.0159,0.0163


,USA,Bonds,World,EM
1,0.0483,0.0088,-0.0085,0.0665
5,0.0157,0.0346,-0.0272,0.0097
10,0.108,0.0014,0.0503,0.033
25,0.0911,0.0091,0.0757,-0.0396


In [69]:
n_posterior_samples = 1000
n_samples = 10
n_steps = 100
scenarios = zeros(K, n_steps, n_posterior_samples)


for s in 1:n_posterior_samples
    
    μ = collect(Iterators.flatten(mean(group(chain, :μ)[s,:,:].value.data, dims=3)))
    σ = collect(Iterators.flatten(mean(group(chain, :σ)[s,:,:].value.data, dims=3)))
    L = collect(Iterators.flatten(mean(group(chain, "L.L")[s,:,:].value.data, dims=3)))
    Σ = est_covariance(σ, L)

    for t in 1:n_steps
    scenarios[:,t,s]= rand(MvNormal(μ, Σ), 1)
    end

end

#= for k in 1:K
    display(round.(quantile(scenarios[k,1,:], [.025, .25, .5, .75, .975]), digits=2))
end =#
ret_in_years = cum_returns_in_periods(scenarios, [1,5, 15, 25], freq, true)
print_scenarios_summary(ret_in_years)

,USA,Bonds,World,EM
1,0.0749,0.0242,0.0268,0.0641
5,0.0712,0.0245,0.0267,0.0572
10,0.072,0.0235,0.0276,0.0567
25,0.0708,0.0242,0.0256,0.0563


,USA,Bonds,World,EM
1,0.1434,0.0858,0.1619,0.2417
5,0.0643,0.0405,0.0747,0.111
10,0.0404,0.0243,0.0447,0.0674
25,0.0321,0.02,0.0357,0.0539


,USA,Bonds,World,EM
1,-0.0186,0.0418,0.0264,-0.0317
5,0.0846,-0.1395,-0.0454,-0.0911
10,-0.0254,-0.1286,-0.0169,0.0547
25,-0.0101,-0.0619,-0.0353,0.0842


,USA,Bonds,World,EM
1,-0.1106,0.1789,-0.1562,-0.0159
5,-0.1822,0.2226,-0.1636,-0.2142
10,-0.1176,-0.1489,-0.0105,-0.2393
25,-0.1838,0.0008,-0.0345,-0.2433


In [76]:
T, K = size(real_returns)

Y, X = prepare_var_data(values(real_returns), 1, Matrix{Float64}(undef, 0, 0), true)

model = var_model(Y, X, 1)

# Sample from the posterior
chain_var = sample(model, NUTS(), MCMCThreads(), 1000, 4; discard_initial=1000)

# Summarize results
display(summarystats(chain_var))
display(quantile(chain_var))

Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.00625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 9.765625e-5
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
Sampling (4 threads)  25%|████████                      |  ETA: 0:01:13
Sampling (4 threads)  50%|████████████████              |  ETA: 0:01:29
Sampling (4 threads)  75%|███████████████████████       |  ETA: 0:00:30
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:01:33
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:01:33


Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

      μ_β[1]    0.0159    0.0065    0.0001   2987.0463   3204.9784    1.0014   ⋯
      μ_β[2]    0.0044    0.0041    0.0001   4731.8018   3412.9496    1.0011   ⋯
      μ_β[3]    0.0054    0.0074    0.0001   2709.5306   3177.3650    1.0013   ⋯
      μ_β[4]    0.0135    0.0110    0.0002   2733.5594   2936.7791    1.0007   ⋯
      μ_β[5]    0.1013    0.1341    0.0030   1938.0573   2724.7665    1.0009   ⋯
      μ_β[6]    0.1160    0.0848    0.0016   2682.3778   2884.0420    1.0006   ⋯
      μ_β[7]    0.0408    0.1549    0.0037   1744.7698   2670.6931    1.0012   ⋯
      μ_β[8]   -0.1693    0.2290    0.0053   1889.1612   2601.4325    1.0002   ⋯
      μ_β[9]    0.0392    0.1501    0.0031   2354.4078   2819.1958    1.0003   ⋯
     μ_β[10]    0.0264    0.0953    0.0016   3553.6882   2765.0846    1.0015   ⋯
     μ_β

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

      μ_β[1]    0.0032    0.0115    0.0158    0.0203    0.0287
      μ_β[2]   -0.0036    0.0016    0.0044    0.0072    0.0126
      μ_β[3]   -0.0091    0.0003    0.0053    0.0105    0.0196
      μ_β[4]   -0.0088    0.0062    0.0135    0.0208    0.0353
      μ_β[5]   -0.1568    0.0112    0.0994    0.1883    0.3668
      μ_β[6]   -0.0539    0.0589    0.1178    0.1751    0.2795
      μ_β[7]   -0.2612   -0.0658    0.0380    0.1454    0.3462
      μ_β[8]   -0.6210   -0.3175   -0.1706   -0.0175    0.2783
      μ_β[9]   -0.2513   -0.0643    0.0403    0.1401    0.3377
     μ_β[10]   -0.1626   -0.0355    0.0290    0.0897    0.2138
     μ_β[11]   -0.4212   -0.1945   -0.0783    0.0428    0.2582
     μ_β[12]   -0.3571   -0.0114    0.1553    0.3231    0.6495
     μ_β[13]   -0.2113   -0.0527    0.0387    0.1244    0.2874
     μ_β[14]   -0.2240   -0.1210   -0.0656

In [124]:
n_assets = 4
p =1
betas = reshape(mean(group(chain_var, :μ_β)).nt.mean, n_assets, n_assets * p + 1)
sigmas = mean(group(chain_var, :σ)).nt.mean
L = mean(group(chain_var, "L.L")).nt.mean
Σ = est_covariance(sigmas, L)

initial_returns = values(real_returns)[end, :]

K= n_assets
n_samples = 10000
n_steps = 25 * freq
scenarios = zeros(K, n_steps, n_samples)

predictors0 = vcat(one(Float64), initial_returns)

scenarios[:,1,:] = rand(MvNormal(betas*predictors0, Σ), n_samples)

for s in 1:n_samples
    for t in 2:n_steps
        predictors = vcat(one(Float64), scenarios[:,t-1,s])
        scenarios[:,t,s] = rand(MvNormal(betas*predictors, Σ))
    end
end

ret_in_years = cum_returns_in_periods(scenarios, [1,5, 15, 25], freq, true)
print_scenarios_summary(ret_in_years)
for a in 1:4
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], assets[a])
end

,USA,Bonds,World,EM
1,0.0739,0.0217,0.0305,0.0507
5,0.0719,0.0234,0.0257,0.0518
10,0.0723,0.0239,0.0252,0.0527
25,0.0724,0.0239,0.0251,0.0527


,USA,Bonds,World,EM
1,0.1587,0.0907,0.1776,0.2628
5,0.0731,0.0415,0.0821,0.1188
10,0.0423,0.024,0.0473,0.0689
25,0.033,0.0188,0.0368,0.0535


,USA,Bonds,World,EM
1,-0.011,0.0157,-0.0462,0.0173
5,0.0003,0.0104,0.0499,0.026
10,0.0346,-0.0355,0.0508,0.0339
25,0.045,-0.0251,0.0184,-0.006


,USA,Bonds,World,EM
1,-0.0473,-0.015,-0.0459,-0.0709
5,-0.0746,0.0288,-0.0527,-0.0233
10,-0.0119,0.0633,0.0007,0.0017
25,-0.015,-0.0095,-0.0726,-0.0406


,0.03,0.25,0.5,0.75,0.97
1,-0.2258,-0.0329,0.0735,0.1797,0.3747
2,-0.0632,0.0222,0.0715,0.1218,0.2082
3,-0.0064,0.0436,0.0721,0.1006,0.1522
4,0.0102,0.0503,0.0719,0.0941,0.1352


,0.03,0.25,0.5,0.75,0.97
1,-0.147,-0.0391,0.0215,0.0823,0.1917
2,-0.0541,-0.0049,0.0233,0.0514,0.1013
3,-0.0212,0.0078,0.024,0.04,0.0691
4,-0.0118,0.0112,0.024,0.0364,0.0593


,0.03,0.25,0.5,0.75,0.97
1,-0.3049,-0.0881,0.0335,0.1519,0.3641
2,-0.127,-0.0312,0.0252,0.0812,0.1822
3,-0.0624,-0.0072,0.0247,0.0575,0.1138
4,-0.0431,-0.0005,0.0247,0.0502,0.0939


,0.03,0.25,0.5,0.75,0.97
1,-0.4338,-0.1301,0.0482,0.2285,0.5431
2,-0.1684,-0.0302,0.0522,0.132,0.2814
3,-0.0746,0.0059,0.0521,0.0995,0.1845
4,-0.0463,0.0168,0.0523,0.0894,0.1541


#### TODO
- est cumr returns, 1, 5, 30
- display table mean, std, skewness, kurtosis, for every series, every time step
- display correlation matrix for every time step


In [53]:
freq = 4
n_periods = 4

ret_in_years = cum_returns_in_periods(scenarios, [1,5, 15, 25], 4, true)

means = zeros(4, 4)
stds = zeros(4, 4)
skew = zeros(4, 4)
kurt = zeros(4, 4)

for a in 1:4
   
    means[:,a] = mean(ret_in_years[a,:,:], dims=2)
    stds[:,a] = std(ret_in_years[a, :,:], dims=2)
    
    for t in 1:n_periods
      skew[t,a] = skewness(ret_in_years[a, t,:])
    end
    
    for t in 1:n_periods
      kurt[t,a] = kurtosis(ret_in_years[a, t,:])
    end 

end

pretty_table(round.(means, digits=4), backend = Val(:html), header=[:USA, :Bonds, :World, :EM], row_labels= ["1", "5", "10", "25"], title="Means")
pretty_table(round.(stds, digits=4), backend = Val(:html), header=[:USA, :Bonds, :World, :EM], row_labels= ["1", "5", "10", "25"], title="Standard devations")
pretty_table(round.(skew, digits=4), backend = Val(:html), header=[:USA, :Bonds, :World, :EM], row_labels= ["1", "5", "10", "25"], title="Skewness")
pretty_table(round.(kurt, digits=4), backend = Val(:html), header=[:USA, :Bonds, :World, :EM], row_labels= ["1", "5", "10", "25"], title="Kurtosis") 



,USA,Bonds,World,EM
1,0.077,0.0192,0.0291,0.0581
5,0.0736,0.023,0.0282,0.0602
10,0.0717,0.0225,0.0263,0.0603
25,0.0709,0.0226,0.0257,0.0591


,USA,Bonds,World,EM
1,0.1467,0.0917,0.1617,0.2388
5,0.0658,0.0406,0.0745,0.1108
10,0.0407,0.0232,0.0465,0.0668
25,0.0318,0.0186,0.0364,0.0525


,USA,Bonds,World,EM
1,-0.108,-0.0043,-0.1885,-0.0494
5,0.0093,0.0043,-0.0308,0.0308
10,0.1327,0.0335,-0.018,-0.0134
25,0.006,0.0723,-0.0856,0.0005


,USA,Bonds,World,EM
1,-0.0034,0.095,0.2343,0.1633
5,-0.0343,-0.0658,-0.2059,0.0825
10,0.4249,0.0504,0.0329,-0.1988
25,0.0079,0.0551,0.2249,0.196


In [56]:
print_scenarios_summary(ret_in_years)

,USA,Bonds,World,EM
1,0.077,0.0192,0.0291,0.0581
5,0.0736,0.023,0.0282,0.0602
10,0.0717,0.0225,0.0263,0.0603
25,0.0709,0.0226,0.0257,0.0591


,USA,Bonds,World,EM
1,0.1467,0.0917,0.1617,0.2388
5,0.0658,0.0406,0.0745,0.1108
10,0.0407,0.0232,0.0465,0.0668
25,0.0318,0.0186,0.0364,0.0525


,USA,Bonds,World,EM
1,-0.108,-0.0043,-0.1885,-0.0494
5,0.0093,0.0043,-0.0308,0.0308
10,0.1327,0.0335,-0.018,-0.0134
25,0.006,0.0723,-0.0856,0.0005


,USA,Bonds,World,EM
1,-0.0034,0.095,0.2343,0.1633
5,-0.0343,-0.0658,-0.2059,0.0825
10,0.4249,0.0504,0.0329,-0.1988
25,0.0079,0.0551,0.2249,0.196


In [30]:
scenarios[1,:,:]

100×1000 Matrix{Float64}:
  0.125684      0.0650308     0.134782    …  -0.16996     -0.0952812
  0.00562433    0.121378      0.0942498       0.00369416  -0.00590361
 -0.148834      0.0367806     0.0828202       0.0237358   -0.0491385
  0.0421623    -0.0196614     0.0787062       0.00389176  -0.147776
  0.097869      0.179976     -0.00288928     -0.0844872    0.101058
 -0.0279182     0.077113      0.0548067   …   0.0899094    0.0310203
 -0.0683473     0.0266058    -0.0412369      -0.0941391    0.0149195
  0.0922975     0.163843      0.111282       -0.0048513   -0.0397223
  0.00738061    0.0047806    -0.0417002       0.0850779   -0.0560816
 -0.000131958  -0.000909428  -0.0290407       0.0459042   -0.177647
  ⋮                                       ⋱               
 -0.0715031     0.213259     -0.0666782       0.0858154    0.0664962
 -0.104314     -0.00397005    0.0220202       0.031515    -0.0426831
 -0.0784141    -0.00290314    0.0138105       0.133309    -0.0806907
  0.0724462     0.01

In [28]:
period_ret = sum_returns_between_periods(scenarios[a, :, :], [1, 5*4, 15*4, 25*4])
mean(period_ret, dims=2) ./ [5, 10, 10]

3×1 Matrix{Float64}:
 0.07364009260999863
 0.07293922533126387
 0.07159645083010083

In [13]:
#= for a in 1:4
    cum_ret  = cumsum(scenarios[a, :, :], dims=1)
    ret_in_years =  cum_ret[[1 * freq, freq * 5, freq * 10, freq * 25], :] ./ [1, 5, 10, 25]

    skew[:,a] = skew(ret_in_years, dims=2)
end
 =#